### 特征匹配

# BFMatcher 解释

`cv2.BFMatcher` 是 OpenCV 库中的一个类，用于暴力匹配（Brute Force Matching）特征描述子。暴力匹配是一种直接比较描述子的匹配方法，通过计算描述子之间的距离来找到最佳匹配对。

## 参数解释

### `crossCheck`

- `crossCheck=True`：启用交叉检查。在这种模式下，匹配对必须满足两个条件：
  1. 描述子 A 的最佳匹配是描述子 B。
  2. 描述子 B 的最佳匹配是描述子 A。

  这种模式可以减少错误匹配，但可能会丢失一些正确的匹配对。

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
img1 = cv2.imread('photoOpencv/box.png', 0)
img2 = cv2.imread('photoOpencv/box_in_scene.png', 0)

In [5]:
def cv_show(name,img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
cv_show('img1',img1)

In [7]:
cv_show('img2',img2)

In [8]:
sift = cv2.xfeatures2d.SIFT_create()

In [9]:
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

In [10]:
bf = cv2.BFMatcher(crossCheck=True)

#### 1对1的匹配
# bf.match 方法解析

`bf.match` 方法是 OpenCV 库中 `cv2.BFMatcher` 类提供的一个函数，用于对两个集合的特征描述子进行暴力匹配（Brute Force Matching）。该方法通过计算描述子之间的距离来找到最佳匹配对。

## 方法签名

```python
matches = bf.match(queryDescriptors, trainDescriptors)
```

## 参数解释

- `queryDescriptors`：第一幅图像的特征描述子，通常是一个 NumPy 数组，每一行对应一个关键点的描述子。
- `trainDescriptors`：第二幅图像的特征描述子，也是一个 NumPy 数组，每一行对应一个关键点的描述子。

## 返回值

- `matches`：一个包含 DMatch 对象的列表，每个 DMatch 对象包含以下信息：
  - `distance`：描述子之间的距离，距离越小表示匹配越好。
  - `queryIdx`：查询描述子（第一幅图像中的描述子）的索引。
  - `trainIdx`：训练描述子（第二幅图像中的描述子）的索引。
  - `imgIdx`：训练图像的索引（在多图像匹配时使用）。

## 匹配过程

`bf.match` 方法的匹配过程如下：

1. **计算距离**：对于每个查询描述子，计算其与所有训练描述子之间的距离。
2. **找到最佳匹配**：选择距离最小的训练描述子作为最佳匹配对。
3. **返回匹配结果**：将所有匹配对作为 `DMatch` 对象存储在列表中并返回。

In [11]:
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)

In [12]:
img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches[:10], None,flags=2)

In [13]:
cv_show('img3',img3)

# bf.knnMatch 方法解析

`bf.knnMatch` 方法是 OpenCV 库中 `cv2.BFMatcher` 类提供的一个函数，用于对两个集合的特征描述子进行 K 最近邻匹配（K-Nearest Neighbors Matching）。该方法通过计算描述子之间的距离来找到 K 个最佳匹配对。

## 匹配过程

`bf.knnMatch` 方法的匹配过程如下：

1. **计算距离**：对于每个查询描述子，计算其与所有训练描述子之间的距离。
2. **找到 K 个最佳匹配**：选择距离最小的 K 个训练描述子作为最佳匹配对。
3. **返回匹配结果**：将所有匹配对作为 `DMatch` 对象存储在列表中并返回。
## 参数解释

- `queryDescriptors`：第一幅图像的特征描述子，通常是一个 NumPy 数组，每一行对应一个关键点的描述子。
- `trainDescriptors`：第二幅图像的特征描述子，也是一个 NumPy 数组，每一行对应一个关键点的描述子。
- `k`：每个查询描述子寻找的最佳匹配个数。

## 返回值

- `matches`：一个包含 DMatch 对象列表的列表，每个子列表包含 K 个 DMatch 对象。每个 DMatch 对象包含以下信息：
  - `distance`：描述子之间的距离，距离越小表示匹配越好。
  - `queryIdx`：查询描述子（第一幅图像中的描述子）的索引。
  - `trainIdx`：训练描述子（第二幅图像中的描述子）的索引。
  - `imgIdx`：训练图像的索引（在多图像匹配时使用）。


## Ratio Test

由于 KNN 匹配会返回多个匹配结果，因此通常会应用比例测试（Ratio Test）来筛选出好的匹配对。比例测试的步骤如下：

1. 对于每个查询描述子，保留其与训练描述子的最近邻和次近邻匹配。
2. 计算最近邻和次近邻匹配的距离比值。
3. 如果比值小于设定的阈值（例如 0.75），则保留这个匹配对，否则丢弃。

In [14]:
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

In [15]:
good = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good.append([m])

In [16]:
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=2)

In [17]:
cv_show('img3',img3)